In [1]:
print('hello')

hello


In [2]:
!which python

/opt/conda/bin/python


In [3]:
!java -version

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [4]:
!which java

/usr/bin/java


In [9]:
pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
#SPARK_HOME
import os
os.environ.get('SPARK_HOME')

'/usr/local/spark'

In [18]:
#JAVA_HOME
os.environ.get('JAVA_HOME')

In [13]:
#PYTHON_HOME
os.environ.get('PYTHONPATH')

'/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python:'

In [19]:
import pyspark

In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()
#SparkContext.SparkSession

In [24]:
spark

In [25]:
spark.stop()

In [26]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [27]:
spark

In [30]:
data = [('Alice',1),('Bob',2),('Charlie',3)]
type(data)

list

In [35]:
#DataFrame 객체(분산객체)를 생성 - 판다스의 데이터프레임이 아님
data1 = spark.createDataFrame(data,['Name','Value'])
data1[0][0]

Column<'Name[0]'>

In [40]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [52]:
data1.filter(data1.Name == 'Bob').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [54]:
data1.filter(data1.Value >= 2).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [56]:
data1.createOrReplaceTempView('people')

In [59]:
spark.sql('SELECT * FROM people').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [62]:
spark.sql('SELECT * FROM people WHERE Name = "Bob"').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [63]:
spark.sql('SELECT * FROM people WHERE Value > 2').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [64]:
spark.sql('SELECT * FROM people WHERE Name LIKE "%l%"').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|Charlie|    3|
+-------+-----+



# RDD 객체 생성

In [41]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [42]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:289

In [43]:
data1

DataFrame[Name: string, Value: bigint]

In [45]:
rdd.take(5) #rdd 객체를 출력하는 함수 - n개를 지정

[1, 2, 3, 4, 5]

In [46]:
#map 연산 : rdd 값의로 연산
squared_rdd = rdd.map(lambda x:x*x)
squared_rdd

PythonRDD[25] at RDD at PythonRDD.scala:53

In [50]:
squared_rdd.take(3)

[1, 4, 9]

In [51]:
rdd.take(3)

[1, 2, 3]

In [49]:
squared_rdd.collect()

[1, 4, 9, 16, 25]

# MLlib

In [65]:
from pyspark.ml.regression import LinearRegression

In [66]:
import numpy as np

In [67]:
from pyspark.ml.feature import VectorAssembler

In [70]:
data_age = [('Alice',25),('Bob',30),('Charlie',33)]
data2 = spark.createDataFrame(data_age,['Name','Age'])
data2

DataFrame[Name: string, Age: bigint]

In [71]:
assembler = VectorAssembler(inputCols=['Age'],outputCol='features')
vector_df = assembler.transform(data2)
vector_df

DataFrame[Name: string, Age: bigint, features: vector]

In [73]:
lr = LinearRegression(featuresCol='features', labelCol='Age')
model = lr.fit(vector_df)

In [74]:
pred = model.transform(vector_df)
pred

DataFrame[Name: string, Age: bigint, features: vector, prediction: double]

In [75]:
pred.show()

+-------+---+--------+-----------------+
|   Name|Age|features|       prediction|
+-------+---+--------+-----------------+
|  Alice| 25|  [25.0]|24.99999999999993|
|    Bob| 30|  [30.0]|30.00000000000001|
|Charlie| 33|  [33.0]|33.00000000000006|
+-------+---+--------+-----------------+



In [77]:
spark.stop()

# Streaming

In [76]:
from pyspark.sql.functions import explode, split

In [78]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [ ]:
lines = spark.readStream.format('socket')\
             .option('host','localhost')\
             .option('port',9999)\
             .load() #STREAMING DATA

In [ ]:
lines.select(explode(split(lines.value, ' '))).alias('word')

In [79]:
spark.stop()